## 상권데이터 전처리


#### 필요한 패키지 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import matplotlib.font_manager as fm

# ✅ 직접 찾은 폰트 경로 지정
font_path = "/content/drive/MyDrive/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothic.ttf"
font_prop = fm.FontProperties(fname=font_path)
print("Okay, let's get started!")

Okay, let's get started!


### 좌표 전환 및 거리 계산을 위한 전처리
- 대규모 점포: '쇼핑센터', '백화점', '대형마트' 등만 추출 후 좌표 -> 위경도
- 휴게음식: 편의점, 카페 등만 추출
- 역은 위경도를 포함함으로 위경도만 추출

#### 모든 위치 데이터는 위경도로 전환

#### 데이터 로드

In [ ]:
depart_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/머신러닝 변수/서울시 대규모점포_도로명.csv", encoding='cp949')
univ_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/머신러닝 변수/서울시 대학교_도로명.csv", encoding='cp949')
subway_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/머신러닝 변수/서울시 지하철 위치.csv", encoding='cp949')


In [ ]:
depart_df_col = depart_df.columns
univ_df_col = univ_df.columns
subway_df_col = subway_df.columns

print(depart_df_col, "\n\n",univ_df_col,"\n\n", subway_df_col)

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '점포구분명'],
      dtype='object') 

 Index(['키값', '연도', '학교종류', '학교명', '본분교', '학교상태', '설립', '우편번호', '주소', '도로명주소',
       '행정시', '행정구', '행정동', '전화번호', '팩스번호', '홈페이지'],
      dtype='object') 

 Index(['연번', '호선', '고유역번호(외부역코드)', '역명', '위도', '경도', '작성일자'], dtype='object')


#### 대형점포 데이터 전처리

In [ ]:
depart_df.shape

(1021, 26)

In [ ]:
len(depart_df_col)

26

In [ ]:
depart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021 entries, 0 to 1020
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   개방자치단체코드  1021 non-null   int64  
 1   관리번호      1021 non-null   int64  
 2   인허가일자     1021 non-null   object 
 3   인허가취소일자   7 non-null      float64
 4   영업상태코드    1021 non-null   int64  
 5   영업상태명     1021 non-null   object 
 6   상세영업상태코드  1021 non-null   object 
 7   상세영업상태명   1013 non-null   object 
 8   폐업일자      188 non-null    object 
 9   휴업시작일자    20 non-null     object 
 10  휴업종료일자    20 non-null     object 
 11  재개업일자     35 non-null     object 
 12  전화번호      993 non-null    object 
 13  소재지면적     945 non-null    float64
 14  소재지우편번호   403 non-null    object 
 15  지번주소      900 non-null    object 
 16  도로명주소     840 non-null    object 
 17  도로명우편번호   661 non-null    object 
 18  사업장명      1021 non-null   object 
 19  최종수정일자    1021 non-null   object 
 20  데이터갱신구분   1021 non-null   obje

In [ ]:
depart_df.head()
depart_df['업태구분명'].unique()

array(['쇼핑센터', '구분없음', '그 밖의 대규모점포', '백화점', '대형마트', '복합쇼핑몰', '전문점', '시장',
       nan], dtype=object)

In [ ]:
depart_li = ['쇼핑센터', '백화점', '대형마트']
depart_df = depart_df[depart_df['업태구분명'].isin(depart_li)]

In [ ]:
depart_df.head()
depart_df.reset_index(drop=True, inplace=True)

In [ ]:
depart_df['점포구분명'].value_counts()

,count
점포구분명,
대규모점포,44
준대규모점포,10


In [ ]:
depart_df['폐업일자'].value_counts()


,count
폐업일자,
20190831,2
20030401,2
20110519,2
20220325,1
20081031,1
20140531,1
20100210,1
20140508,1
2023-07-01,1


In [ ]:
depart_df['폐업일자'].isnull().sum()


180

In [ ]:
closed_missing = depart_df[depart_df['폐업일자'].isnull()]
closed_missing.reset_index(drop=True, inplace=True)
closed_missing

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
0,3170000,2013317014207500002,2013-09-06,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 금천구 디지털로 185 (가산동),153-801,마리오아울렛 1관,2023-02-28 20:56:47,U,2022-12-03 00:03:00.0,쇼핑센터,189943.234285,441784.247519,NaN
1,3170000,2012317014207500004,2012-09-13,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 금천구 벚꽃로 266 (가산동),NaN,마리오아울렛 3관,2023-02-28 21:05:56,U,2022-12-03 00:03:00.0,쇼핑센터,189759.713051,441814.115921,NaN
2,3220000,2001322008307500007,1985-07-08,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,"서울특별시 강남구 압구정로 407 (압구정동, 갤러리아백화점)",135-902,갤러리아백화점(동관),2023-03-28 11:23:26,U,2022-12-02 21:00:00.0,백화점,203604.761652,447316.222786,NaN
3,3220000,2001322008307500028,1985-03-18,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,"서울특별시 강남구 압구정로 343 (압구정동, 갤러리아백화점)",135-902,갤러리아백화점(서관),2023-03-28 11:22:24,U,2022-12-02 21:00:00.0,백화점,203470.848439,447369.579852,NaN
4,3230000,2009323013107500001,2009-05-11,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 송파구 충민로 10 (문정동),138-962,가든파이브 툴,2023-01-16 17:51:49,U,2022-12-04 22:06:00.0,쇼핑센터,210480.282019,441833.163235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,3010000,2010301010007500001,1973-04-10,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,"서울특별시 중구 소공로 63 (충무로1가,신세계백화점건물)",NaN,신세계백화점,2024-04-26 11:21:28,U,2023-12-03 22:08:00.0,백화점,198263.908392,450960.762965,NaN
176,3180000,2001318007607500031,2001-09-25,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 영등포구 영중로 125 (영등포동8가),150-038,롯데쇼핑(주) 롯데마트맥스 영등포점,2024-05-28 13:52:43,U,2023-12-04 21:00:00.0,대형마트,191503.386112,447289.548355,NaN
177,3170000,2002317006707500001,2002-06-18,NaN,1,영업/정상,1,정상영업,NaN,2012-01-30,...,서울특별시 금천구 두산로 71 (독산동),153-813,롯데쇼핑(주) 롯데마트맥스 금천점,2024-06-12 09:17:21,U,2023-12-05 23:04:00.0,대형마트,190704.904313,440940.384414,NaN
178,3040000,2008304010307500001,20080822,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 광진구 능동로 92 (자양동),143854,롯데백화점 건대스타시티점,2022-05-13 14:43:47,U,2021-12-04 23:05:00.0,백화점,206217.434726,448506.218744,NaN


In [ ]:
col_depart = ['사업장명','도로명주소','좌표정보(X)','좌표정보(Y)']
closed_missing = closed_missing[col_depart]
closed_missing

,사업장명,도로명주소,좌표정보(X),좌표정보(Y)
0,마리오아울렛 1관,서울특별시 금천구 디지털로 185 (가산동),189943.234285,441784.247519
1,마리오아울렛 3관,서울특별시 금천구 벚꽃로 266 (가산동),189759.713051,441814.115921
2,갤러리아백화점(동관),"서울특별시 강남구 압구정로 407 (압구정동, 갤러리아백화점)",203604.761652,447316.222786
3,갤러리아백화점(서관),"서울특별시 강남구 압구정로 343 (압구정동, 갤러리아백화점)",203470.848439,447369.579852
4,가든파이브 툴,서울특별시 송파구 충민로 10 (문정동),210480.282019,441833.163235
...,...,...,...,...
175,신세계백화점,"서울특별시 중구 소공로 63 (충무로1가,신세계백화점건물)",198263.908392,450960.762965
176,롯데쇼핑(주) 롯데마트맥스 영등포점,서울특별시 영등포구 영중로 125 (영등포동8가),191503.386112,447289.548355
177,롯데쇼핑(주) 롯데마트맥스 금천점,서울특별시 금천구 두산로 71 (독산동),190704.904313,440940.384414
178,롯데백화점 건대스타시티점,서울특별시 광진구 능동로 92 (자양동),206217.434726,448506.218744


In [ ]:
closed_missing.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/department_address.csv', index=False)

In [ ]:
depart_xy = closed_missing[['좌표정보(X)','좌표정보(Y)']]
depart_xy_2 = depart_xy

In [ ]:
# from pyproj import Proj, transform

# # TM 중부원점 좌표계 (EPSG:2097) → WGS84 (EPSG:4326)
# tm_proj = Proj(init='epsg:2097')  # 또는 epsg:5179 (UTM-K)
# wgs84_proj = Proj(init='epsg:4326')  # WGS84 GPS 좌표

# # 예제 좌표 (마리오아울렛 1관)
# x, y = 189943.234285, 441784.247519

# # 좌표 변환
# lon, lat = transform(tm_proj, wgs84_proj, x, y)

# print(f"위도: {lat}, 경도: {lon}")


위도: 37.47818895227105, 경도: 126.88420115343547


/usr/local/lib/python3.11/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.11/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-31-1ea646efef52>:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(tm_proj, wgs84_proj, x, y)


In [ ]:
from pyproj import Transformer

# 🔹 좌표 변환기 생성 (ITRF2000 → WGS84)
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)

# 🔹 좌표 변환 함수
def convert_coords(row):
    x, y = row["좌표정보(X)"], row["좌표정보(Y)"]
    lon, lat = transformer.transform(x, y)
    return pd.Series([lon, lat])

# 🔹 apply()를 이용한 변환 (벡터 연산 적용)
depart_xy_2[["경도", "위도"]] = depart_xy_2.apply(convert_coords, axis=1)

# 🔹 기존 X, Y 좌표 삭제 (필요 없으면 유지)
depart_xy_2.drop(columns=["좌표정보(X)", "좌표정보(Y)"], inplace=True)



KeyError: '좌표정보(X)'

In [ ]:
# 결과 확인
print(depart_xy_2.head())
print(depart_xy.head())

           경도         위도
0  126.887641  36.574362
1  126.885590  36.574629
2  127.040300  36.624259
3  127.038803  36.624741
4  127.117092  36.574798
           경도         위도
0  126.887641  36.574362
1  126.885590  36.574629
2  127.040300  36.624259
3  127.038803  36.624741
4  127.117092  36.574798


In [ ]:
depart_xy.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/department_lon_lat.csv', index=False)

#### 대학교 데이터 전처리

In [ ]:
univ_df.shape

(64, 16)

In [ ]:
univ_df_col

Index(['키값', '연도', '학교종류', '학교명', '본분교', '학교상태', '설립', '우편번호', '주소', '도로명주소',
       '행정시', '행정구', '행정동', '전화번호', '팩스번호', '홈페이지'],
      dtype='object')

In [ ]:
univ_df

,키값,연도,학교종류,학교명,본분교,학교상태,설립,우편번호,주소,도로명주소,행정시,행정구,행정동,전화번호,팩스번호,홈페이지
0,BE_LiST23-0034,2013,일반대학,서울시립대학교,본교,기존,공립,NaN,서울 동대문구 서울시립대로 163 (전농동 90번지),NaN,서울특별시,동대문구,휘경2동,02-6490-6114,02-2210-5576,http://www.uos.ac.kr
1,BE_LiST23-0035,2013,전문대학(3년제),서울여자간호대학교,본교,기존,사립,NaN,서울 서대문구 홍제3동 서울여자간호대학,NaN,서울특별시,서대문구,홍제2동,02-2287-1700,02-395-8018,http://www.snjc.ac.kr
2,BE_LiST23-0036,2013,일반대학,서울여자대학교,본교,기존,사립,NaN,서울특별시 노원구 화랑로 621 서울여자대학교,NaN,서울특별시,노원구,공릉2동,02-970-5114,02-978-7931,http://www.swu.ac.kr
3,BE_LiST23-0037,2013,전문대학(2년제),서일대학교,본교,학교명 변경,사립,NaN,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교,NaN,서울특별시,중랑구,면목제3.8동,02-490-7300,02-493-2576,http://www.seoil.ac.kr
4,BE_LiST23-0038,2013,일반대학,성공회대학교,본교,기존,사립,NaN,서울 구로구 항동 성공회대학교,NaN,서울특별시,구로구,오류2동,02-2610-4142,02-2610-4248,http://www.skhu.ac.kr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,BE_LiST23-0029,2013,교육대학,서울교육대학교,본교,기존,국립,NaN,서울 서초구 서초중앙로 96(서초동 1650),NaN,서울특별시,서초구,서초1동,02-3475-2114,02-581-7711,http://www.snue.ac.kr
60,BE_LiST23-0030,2013,일반대학,서울기독대학교,본교,기존,사립,NaN,서울 은평구 갈현로 4길 26-2호,NaN,서울특별시,은평구,신사1동,02-380-2500,02-380-2535,http://www.scu.ac.kr
61,BE_LiST23-0031,2013,일반대학,서울대학교,본교,기존,국립대법인,NaN,서울 관악구 관악로 1 서울대학교,NaN,서울특별시,관악구,낙성대동,02-880-5114,02-885-5272,http://www.snu.ac.kr
62,BE_LiST23-0032,2013,사이버대학(대학),서울디지털대학교,본교,기존,사립,NaN,서울 마포구 도화동 560번지,NaN,서울특별시,마포구,도화동,02-1644-0982,02-2128-3006,http://www.sdu.ac.kr


In [ ]:
univ_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   키값      64 non-null     object 
 1   연도      64 non-null     int64  
 2   학교종류    64 non-null     object 
 3   학교명     64 non-null     object 
 4   본분교     64 non-null     object 
 5   학교상태    64 non-null     object 
 6   설립      64 non-null     object 
 7   우편번호    0 non-null      float64
 8   주소      64 non-null     object 
 9   도로명주소   0 non-null      float64
 10  행정시     64 non-null     object 
 11  행정구     64 non-null     object 
 12  행정동     64 non-null     object 
 13  전화번호    60 non-null     object 
 14  팩스번호    50 non-null     object 
 15  홈페이지    63 non-null     object 
dtypes: float64(2), int64(1), object(13)
memory usage: 8.1+ KB


In [ ]:
univ_df['도로명주소'].isnull().sum()

64

In [ ]:
col_univ = ['학교명','주소']
univ_df = univ_df[col_univ]
univ_df.reset_index(drop=True, inplace=True)
univ_df

,학교명,주소
0,서울시립대학교,서울 동대문구 서울시립대로 163 (전농동 90번지)
1,서울여자간호대학교,서울 서대문구 홍제3동 서울여자간호대학
2,서울여자대학교,서울특별시 노원구 화랑로 621 서울여자대학교
3,서일대학교,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교
4,성공회대학교,서울 구로구 항동 성공회대학교
...,...,...
59,서울교육대학교,서울 서초구 서초중앙로 96(서초동 1650)
60,서울기독대학교,서울 은평구 갈현로 4길 26-2호
61,서울대학교,서울 관악구 관악로 1 서울대학교
62,서울디지털대학교,서울 마포구 도화동 560번지


In [ ]:
univ_df.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/university_address.csv', index=False)

#### 지하철 데이터 전처리

In [ ]:
subway_df.shape

(276, 7)

In [ ]:
subway_df_col = subway_df.columns
subway_df_col

Index(['연번', '호선', '고유역번호(외부역코드)', '역명', '위도', '경도', '작성일자'], dtype='object')

In [ ]:
col_subway = ['역명', '위도', '경도']
subway_df = subway_df[col_subway]
subway_df.reset_index(drop=True, inplace=True)
subway_df

,역명,위도,경도
0,서울,37.553150,126.972533
1,시청,37.563590,126.975407
2,종각,37.570203,126.983116
3,종로3가,37.570429,126.992095
4,종로5가,37.570971,127.001900
...,...,...,...
271,남한산성입구,37.451568,127.159845
272,단대오거리,37.445057,127.156735
273,신흥,37.440952,127.147590
274,수진,37.437575,127.140936


In [ ]:
subway_df.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/subway_lat_lon.csv', index=False)